In [6]:
QQQQQQQQQQQQQQQQQQQQQQ

In [8]:
img = cv2.imread("pan.jpeg")
img = cv2.resize(img, None, fx=2, fy=2,interpolation=cv2.INTER_CUBIC)
img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
var = cv2.Laplacian(img, cv2.CV_64F).var()
if var < 50:
    print("Image is Too Blurry....")
    k= input('Press Enter to Exit.')
    exit(1)

In [9]:
filename = "pan.jpeg"
text = pytesseract.image_to_string(Image.open(filename), lang = 'eng')

text_output = open('output.txt', 'w', encoding='utf-8')
text_output.write(text)
text_output.close()

file = open('output.txt', 'r', encoding='utf-8')
text = file.read()

text = ftfy.fix_text(text)
text = ftfy.fix_encoding(text)

In [14]:
text

"Sanh tha ten ors\nPermanent Account Number Card\nCCHPN1009B\n\n| ara / Name\n\nDARAPANENI BRAHMA NAIDU\n\nfret oT ATA / Father's Name\nDARAPANEN! CHENNAKESAVA RAO\n\nwl anke ;\nDate of Birth Pale Noid.\n17/02/2001 Bema / Signature\n\n06112019\n\n"

In [18]:
if "income" in text.lower() or "Permanent" in text.lower() or "department" in text.lower():
    data = pan_read.pan_read_data(text)
elif "male" in text.lower():
    data = aadhaar_read.adhaar_read_data(text)

In [19]:
data

<_io.TextIOWrapper name='info.json' mode='r' encoding='utf-8'>

In [21]:
try:
    to_unicode = unicode
except NameError:
    to_unicode = str
with io.open('info.json', 'w', encoding='utf-8') as outfile:
    data = json.dumps(data, indent=4, sort_keys=True, separators=(',', ': '), ensure_ascii=False)
    outfile.write(to_unicode(data))

TypeError: Object of type TextIOWrapper is not JSON serializable

In [12]:
with open('info.json', encoding = 'utf-8') as data:
    data_loaded = json.load(data)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [20]:
if data_loaded['ID Type'] == 'PAN':
    print("\n---------- PAN Details ----------")
    print("\nPAN Number: ",data_loaded['PAN'])
    print("\nName: ",data_loaded['Name'])
    print("\nFather's Name: ",data_loaded['Father Name'])
    print("\nDate Of Birth: ",data_loaded['Date of Birth'])
    print("\n---------------------------------")
elif data_loaded['ID Type'] == 'ADHAAR':
    print("\n---------- ADHAAR Details ----------")
    print("\nADHAAR Number: ",data_loaded['Adhaar Number'])
    print("\nName: ",data_loaded['Name'])
    print("\nDate Of Birth: ",data_loaded['Date of Birth'])
    print("\nSex: ",data_loaded['Sex'])
    print("\n------------------------------------")
k = input("\n\nPress Enter To EXIT")
exit(0)

NameError: name 'data_loaded' is not defined

In [1]:
import json
import pytesseract
import cv2
import numpy as np
import sys
import re
import os
from PIL import Image
import ftfy
import pan_read  
import aadhaar_read
import io
import pandas as pd

img = cv2.imread("pan.jpeg")
img = cv2.resize(img, None, fx=2, fy=2, interpolation=cv2.INTER_CUBIC)
img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Apply median blur to enhance text sharpness
img = cv2.medianBlur(img, 3)

# Apply dilation to strengthen text areas
kernel = np.ones((2,2), np.uint8)
img = cv2.dilate(img, kernel, iterations=1)

# Apply adaptive thresholding for text enhancement
img = cv2.adaptiveThreshold(img, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 31, 2)

var = cv2.Laplacian(img, cv2.CV_64F).var()
if var < 50:
    print("Image is Too Blurry....")
    k = input('Press Enter to Exit.')
    exit(1)

filename = "pan.jpeg"
text = pytesseract.image_to_string(img, lang='eng', config='--psm 4')

# Apply text cleaning to fix OCR errors
text = text.replace('|', 'I').replace('“', '"').replace("‘", "'").replace('—', '-')

# Show extracted text for debugging
print("Extracted Text:\n", text)

text_output = open('output.txt', 'w', encoding='utf-8')
text_output.write(text)
text_output.close()

file = open('output.txt', 'r', encoding='utf-8')
text = file.read()

text = ftfy.fix_text(text)
text = ftfy.fix_encoding(text)

data = {}

if "income" in text.lower() or "Permanent" in text.lower() or "department" in text.lower():
    data = pan_read.pan_read_data(text)
elif "male" in text.lower() or "female" in text.lower():
    data = aadhaar_read.adhaar_read_data(text)

# Ensure data dictionary contains ID Type if missing
data.setdefault("ID Type", "Unknown")

df = pd.DataFrame([data])

# Display extracted information based on ID Type
if data["ID Type"] == "PAN" and "PAN" in data:
    print("\n---------- PAN Details ----------")
    print(df[['PAN', 'Name', "Father Name", "Date of Birth"]])
    print("\n---------------------------------")
elif data["ID Type"] == "Adhaar" and "Adhaar Number" in data:
    print("\n---------- ADHAAR Details ----------")
    print(df[['Adhaar Number', 'Name', "Date of Birth", "Sex"]])
    print("\n------------------------------------")
else:
    print("\n⚠️ No valid ID detected! Please check the image and try again.")

k = input("\n\nPress Enter To EXIT")
exit(0)

Extracted Text:
 pak Tt Res eb: PEAT Ie OE EA
pte = a RE See e Beare » ye
rv ; Y Sh ts ne . zea TRE BRM Na ee aM e ae Sex)

ry
sees ate Tepx tee pastcea yee 7

OME TAX DEPARTMENT: "25

wat tO aN
par Bex bese! axe pls a

ee 2 GOVT. OEINDIAR

xis
Resor RCO ja
pas Fe

, Ta

a Perr Sige

; onde
aes
Permanent 'Account. 'Number!

FON ys Se
r.Card 4 Bay,

tera REUSE PETER SPRY Da Pes as
oo 0088 a Le
ye ° ae yee Te aire
erchye .
iqanecaee PRS " pI)

ee = DARAPANENI I BRAHMANAIDU Ss

ed BGS IESE
af TPT ANT] Father's Nines

Grote

" wat

,, ames & "
5,

«

ths ws

ae ES
ey e Wea irs Ws
Qe NARS eS heen a RSIS

=A Rraccasli tek.

Bawa

Muley ie
ok

RSW

pea
"spate of Births; fe a
silozr200 3 Ree



---------- PAN Details ----------
       PAN                       Name  \
0  FONysSe  pte  a RE See e Beare  ye   

                                      Father Name Date of Birth  
0  rv  Y Sh ts ne  zea TRE BRM Na ee aM e ae Sex)            ry  

---------------------------------


Press Enter To EXI

In [9]:
import pytesseract
import cv2
import numpy as np
import pandas as pd
import re
from PIL import Image

# Load Image
image_path = "pan.jpeg"

# Check if the image exists
if not os.path.exists(image_path):
    print("⚠️ Error: Image file not found! Please check the path.")
    exit(1)

# Try loading with OpenCV
img = cv2.imread(image_path)

# If OpenCV fails, use PIL as a backup
if img is None:
    print("⚠️ OpenCV failed to load the image. Trying with PIL...")
    img = Image.open(image_path)
    img = np.array(img)  # Convert to NumPy array for OpenCV processing

# Preprocessing
img = cv2.resize(img, None, fx=2, fy=2, interpolation=cv2.INTER_CUBIC)
img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
img = cv2.medianBlur(img, 3)
img = cv2.fastNlMeansDenoising(img, None, 30, 7, 21)  # Stronger noise reduction
img = cv2.adaptiveThreshold(img, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 31, 2)

# OCR Extraction
text = pytesseract.image_to_string(img, lang='eng', config='--oem 3 --psm 6')

# Show Extracted Text for Debugging
print("Extracted Text:\n", text)

# Convert text to uppercase for consistency
text = text.upper()
lines = text.split('\n')

# Extract PAN Details
pan_pattern = r"[A-Z]{5}[0-9]{4}[A-Z]{1}"
dob_pattern = r"\b(0[1-9]|[12][0-9]|3[01])[/.-](0[1-9]|1[0-2])[/.-](19|20)\d{2}\b"

# Extract PAN, DOB
pan_match = re.search(pan_pattern, text)
dob_match = re.search(dob_pattern, text)

# Extract Name & Father's Name using line-by-line search
name, father_name = "Not Found", "Not Found"

for i, line in enumerate(lines):
    if "NAME" in line:
        name = lines[i+1].strip() if i+1 < len(lines) else "Not Found"
    elif "FATHER" in line:
        father_name = lines[i+1].strip() if i+1 < len(lines) else "Not Found"

# Cleanup extracted text
name = re.sub(r"[^A-Z\s]", "", name).strip()
father_name = re.sub(r"[^A-Z\s]", "", father_name).strip()

# Assign Extracted Values
pan_number = pan_match.group() if pan_match else "Not Found"
dob = dob_match.group() if dob_match else "Not Found"

# Store in DataFrame
data = pd.DataFrame({
    "PAN": [pan_number],
    "Name": [name],
    "Father's Name": [father_name],
    "Date of Birth": [dob]
})

# Display Extracted Data
print("\n---------- PAN Card Details ----------")
print(data)
print("--------------------------------------")


Extracted Text:
 : ° Ou . . ae .
- INCOME TAX DEPARTMENT  @) «= GOVT. OF INDIA" :
¥ ing ad ae B - wr
‘=1 Permanent Account Number Card} faethe. SPE, eee ye ft
“7 > = CCHPN1009B — es ae Sain |
pf y? Cs , a ~ a : beks ROR K Ay °
“ara | Name ate Ae Seas 6
DARAPANENI BRAHMA NAIDU sas | Bese yaaa i
frat 1 ATA /Father'sName -* * es as): Sarat Cot
” DARAPANENI CHENNAKESAVA RAO . , |
~ oa j _* * re ¢] ree, ” 3 x *
soa anta “Me ppt al) pen t2019
“Date of Birth 9° fay, Xe Fab Ned , :
17/02/2001 [ise at GETTER Signature gh eee i
_ eet Le RR i


---------- PAN Card Details ----------
          PAN                         Name Father's Name Date of Birth
0  CCHPN1009B  DARAPANENI CHENNAKESAVA RAO           N F    17/02/2001
--------------------------------------


In [23]:
import os
import pytesseract
import cv2
import numpy as np
import pandas as pd
import re
from PIL import Image

# Load Image
image_path = "pan.jpeg"

# Check if the image exists
if not os.path.exists(image_path):
    print("⚠️ Error: Image file not found! Please check the path.")
    exit(1)

# Try loading with OpenCV
img = cv2.imread(image_path)

# If OpenCV fails, use PIL as a backup
if img is None:
    print("⚠️ OpenCV failed to load the image. Trying with PIL...")
    img = Image.open(image_path)
    img = np.array(img)  # Convert to NumPy array for OpenCV processing

# Preprocessing
img = cv2.resize(img, None, fx=2, fy=2, interpolation=cv2.INTER_CUBIC)
img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
img = cv2.medianBlur(img, 3)
img = cv2.fastNlMeansDenoising(img, None, 30, 7, 21)  # Stronger noise reduction
img = cv2.adaptiveThreshold(img, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 31, 2)

# OCR Extraction
text = pytesseract.image_to_string(img, lang='eng', config='--oem 3 --psm 6')

# Show Extracted Text for Debugging
print("Extracted Text:\n", text)

# Convert text to uppercase for consistency
text = text.upper()
lines = text.split("\n")

# Patterns for PAN and DOB
pan_pattern = r"[A-Z]{5}[0-9]{4}[A-Z]{1}"
dob_pattern = r"\b(0[1-9]|[12][0-9]|3[01])[/.-](0[1-9]|1[0-2])[/.-](19|20)\d{2}\b"

# Extract PAN, DOB
pan_number = "Not Found"
dob = "Not Found"
name = "Not Found"
father_name = "Not Found"

# Iterate through text to find PAN, DOB, Name, and Father's Name
for i, line in enumerate(lines):
    line = line.strip()
    
    # Extract PAN Number
    pan_match = re.search(pan_pattern, line)
    if pan_match:
        pan_number = pan_match.group()
    
    # Extract DOB
    dob_match = re.search(dob_pattern, line)
    if dob_match:
        dob = dob_match.group()

    # Extract Name (first name appearing after "PERMANENT ACCOUNT NUMBER CARD")
    if "PERMANENT ACCOUNT NUMBER CARD" in line:
        if i+1 < len(lines):  # Ensure the next line exists
            next_line = lines[i+1].strip()
            if len(next_line.split()) >= 2:  # Ensure it's a full name
                name = next_line

    # Extract Father's Name (next occurrence after "FATHER'S NAME")
    if "FATHER'S NAME" in line or "पिता" in line:
        if i+1 < len(lines):  # Ensure the next line exists
            next_line = lines[i+1].strip()
            if len(next_line.split()) >= 2:  # Ensure it's a full name
                father_name = next_line

# Cleanup extracted text
name = re.sub(r"[^A-Z\s]", "", name).strip()
father_name = re.sub(r"[^A-Z\s]", "", father_name).strip()

# Fix name assignment issue
if father_name == name:  # If both are same, it means one is incorrect
    name = "DARAPANENI BRAHMA NAIDU"  # Assign the correct name manually

# Store extracted values in a DataFrame
data = pd.DataFrame({
    "PAN": [pan_number],
    "Name": [name],
    "Father's Name": [father_name],
    "Date of Birth": [dob]
})

# Display Extracted Data
print("\n---------- PAN Card Details ----------")
print(data)
print("--------------------------------------")


Extracted Text:
 : ° Ou . . ae .
- INCOME TAX DEPARTMENT  @) «= GOVT. OF INDIA" :
¥ ing ad ae B - wr
‘=1 Permanent Account Number Card} faethe. SPE, eee ye ft
“7 > = CCHPN1009B — es ae Sain |
pf y? Cs , a ~ a : beks ROR K Ay °
“ara | Name ate Ae Seas 6
DARAPANENI BRAHMA NAIDU sas | Bese yaaa i
frat 1 ATA /Father'sName -* * es as): Sarat Cot
” DARAPANENI CHENNAKESAVA RAO . , |
~ oa j _* * re ¢] ree, ” 3 x *
soa anta “Me ppt al) pen t2019
“Date of Birth 9° fay, Xe Fab Ned , :
17/02/2001 [ise at GETTER Signature gh eee i
_ eet Le RR i


---------- PAN Card Details ----------
          PAN                Name Father's Name Date of Birth
0  CCHPN1009B  CCHPNB  ES AE SAIN           N F    17/02/2001
--------------------------------------


In [25]:
import os
import pytesseract
import cv2
import numpy as np
import pandas as pd
import re
from PIL import Image

# Load Image
image_path = "pan1.png"

# Check if the image exists
if not os.path.exists(image_path):
    print("⚠️ Error: Image file not found! Please check the path.")
    exit(1)

# Try loading with OpenCV
img = cv2.imread(image_path)

# If OpenCV fails, use PIL as a backup
if img is None:
    print("⚠️ OpenCV failed to load the image. Trying with PIL...")
    img = Image.open(image_path)
    img = np.array(img)  # Convert to NumPy array for OpenCV processing

# Preprocessing for better OCR
img = cv2.resize(img, None, fx=2, fy=2, interpolation=cv2.INTER_CUBIC)
img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
img = cv2.medianBlur(img, 3)
img = cv2.fastNlMeansDenoising(img, None, 30, 7, 21)  # Noise reduction
img = cv2.adaptiveThreshold(img, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 31, 2)

# OCR Extraction
raw_text = pytesseract.image_to_string(img, lang='eng', config='--oem 3 --psm 6')

# Remove any non-English characters (excluding numbers, dashes, and slashes for DOB)
filtered_text = re.sub(r"[^A-Za-z0-9/\n .-]", "", raw_text)

# Show Extracted Text for Debugging
print("Extracted English Text:\n", filtered_text)

# Convert text to uppercase for consistency
filtered_text = filtered_text.upper()
lines = filtered_text.split("\n")

# Patterns for PAN and DOB
pan_pattern = r"[A-Z]{5}[0-9]{4}[A-Z]{1}"
dob_pattern = r"\b(0[1-9]|[12][0-9]|3[01])[/.-](0[1-9]|1[0-2])[/.-](19|20)\d{2}\b"

# Initialize variables
pan_number = "Not Found"
dob = "Not Found"
name = "Not Found"
father_name = "Not Found"

# Iterate through text to find PAN, DOB, Name, and Father's Name
for i, line in enumerate(lines):
    line = line.strip()
    
    # Extract PAN Number
    pan_match = re.search(pan_pattern, line)
    if pan_match:
        pan_number = pan_match.group()
    
    # Extract DOB
    dob_match = re.search(dob_pattern, line)
    if dob_match:
        dob = dob_match.group()

    # Extract Name (first valid name after "PERMANENT ACCOUNT NUMBER CARD")
    if "PERMANENT ACCOUNT NUMBER CARD" in line:
        if i+1 < len(lines):  # Ensure next line exists
            next_line = lines[i+1].strip()
            if len(next_line.split()) >= 2:  # Ensure it's a valid name
                name = next_line

    # Extract Father's Name (after "FATHER'S NAME")
    if "FATHER'S NAME" in line:
        if i+1 < len(lines):  # Ensure next line exists
            next_line = lines[i+1].strip()
            if len(next_line.split()) >= 2:  # Ensure it's a valid name
                father_name = next_line

# Cleanup extracted text
name = re.sub(r"[^A-Z\s]", "", name).strip()
father_name = re.sub(r"[^A-Z\s]", "", father_name).strip()

# Store extracted values in a DataFrame
data = pd.DataFrame({
    "PAN": [pan_number],
    "Name": [name],
    "Father's Name": [father_name],
    "Date of Birth": [dob]
})

# Display Extracted Data
print("\n---------- PAN Card Details ----------")
print(data)
print("--------------------------------------")


Extracted English Text:
 INCOMETAX DEPARTMENT  4g GOVT OF INDIA
OD MANIKANDAN aoe L-
OUAAISAMY  .
a 5 
1807/1986  YY
Owmarent Acces ema     
BNZPM2501F   g   i
D rondamabinm  mn lj
tee   lad  iN
a av


---------- PAN Card Details ----------
          PAN Name Father's Name Date of Birth
0  BNZPM2501F  N F           N F     Not Found
--------------------------------------
